In [ ]:
import xarray as xr
import dask.array as da
import glob
from tqdm import tqdm
import os
DATASET_PATH = "/vast/tm3076/pytorch_learning_tiles"

# Define paths
file_path = f"{DATASET_PATH}/HRS_SST_tiles/agg_cloud_masks_zarr/*"
new_file_path = f"{DATASET_PATH}/HRS_SST_tiles/agg_cloud_masks_zarr_split_t"
# Ensure output directory exists
os.makedirs(new_file_path, exist_ok=True)
# Gather and sort Zarr file paths
zarr_files = sorted(glob.glob(file_path))
# Rechunk and write with progress bar
for zarr_file in tqdm(zarr_files, desc="Rechunking Zarr files"):
    if os.path.exists(os.path.join(new_file_path,os.path.basename(zarr_file))):
        continue
    ds = xr.open_zarr(zarr_file, chunks={})
    ds_rechunked = ds.chunk({"time": 1, "x": 128, "y": 128})
    # REMOVE pre-existing encoding chunks
    for var in ds_rechunked.data_vars:
        ds_rechunked[var].encoding.pop("chunks", None)
    out_path = os.path.join(new_file_path, os.path.basename(zarr_file))
    ds_rechunked.to_zarr(out_path, mode="w", consolidated=True)

Rechunking Zarr files:  58%|█████▊    | 196/337 [2:30:28<1:55:38, 49.21s/it]